# BarbenHeimer with LLaMA 2


![image](https://images.lumacdn.com/cdn-cgi/image/format=auto,fit=cover,dpr=1,quality=75,width=960,height=480/event-covers/87/1f6d4850-0231-4bd9-92e8-af6b45c18d7a)


In the following notebook we'll be discussing Retrieval Augmented Generation - and how to leverage Meta's neweset LLM, LLaMA 2 as the engine!

### Pre-task Work

All we really need to do to get started is to get our prerequisites!

We'll be leveraging `langchain` and `llama 2` today.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [LLaMA 2](https://huggingface.co/blog/llama2)

In [1]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.6/808.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 79.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

In [2]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/oppenheimer.csv

--2023-12-12 10:15:56--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/oppenheimer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80957 (79K) [text/plain]
Saving to: ‘oppenheimer.csv’

oppenheimer.csv     100%[===================>]  79.06K  --.-KB/s    in 0.02s   

2023-12-12 10:15:56 (3.59 MB/s) - ‘oppenheimer.csv’ saved [80957/80957]



In [3]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/barbie.csv

--2023-12-12 10:16:00--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/barbie.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72289 (71K) [text/plain]
Saving to: ‘barbie.csv’

barbie.csv          100%[===================>]  70.59K  --.-KB/s    in 0.02s   

2023-12-12 10:16:00 (3.27 MB/s) - ‘barbie.csv’ saved [72289/72289]



#### Data Parsing

Now that we have our data - let's go ahead and start parsing it into a more usable format for LangChain!

We'll be using the `CSVLoader` for this application.

Check out the docs here:
- [CSVLoader](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [4]:
from langchain.document_loaders.csv_loader import CSVLoader

In [5]:
barbie_loader = CSVLoader(file_path="barbie.csv", source_column="Review_Url")

barbie_data = barbie_loader.load()

In [6]:
len(barbie_data)

125

In [7]:
oppenheimer_loader = CSVLoader(file_path="oppenheimer.csv", source_column="Review_Url")

oppenheimer_data = oppenheimer_loader.load()

In [8]:
len(oppenheimer_data)

150

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

In [10]:
barbie_documents = text_splitter.transform_documents(barbie_data)

In [11]:
len(barbie_documents)

181

In [12]:
oppenheimer_documents = text_splitter.transform_documents(oppenheimer_data)

In [13]:
len(oppenheimer_documents)

186

In [14]:
combined_documents = barbie_documents + oppenheimer_documents

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

### Task 2: Creating an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

#### Selecting Our VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [15]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 98.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [16]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(combined_documents, embedder)

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Now that we've created the VectorStore, we can check that it's working by embedding a query and retrieving passages from our reviews that are close to it.

In [17]:
query = "How is Will Ferrell in this movie?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

: 61
Review_Date: 23 July 2023
Author: agjbull
Rating: 6
Review_Title: Just a little empty
Review: I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow. The film was very disjointed. Ryan Gosling as good as he is seemed to old to play Ken and Will Ferrell ruined every scene he was in. I just didn't get it, it seemed hollow artificial and hackneyed. A waste of some great talent. It was predictable without being reassuring and trying so hard to be woke in the most superficial way in that but trying to tick so many boxes it actually ticked none. Margo Robbie looks beautiful throughout, the costumes and the sets were amazing but the story was way too weak and didn't make much sense at all.
Review_Url: /review/rw9199947/?ref_=tt_urv
: 85
Review_Date: 23 July 2023
Author: hyllus-01262
Rating: 6
Review_Title: Overhyped movie, had its moments though
Review: The first half was pretty enjoyable, fun, light, but it took itse

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [18]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

11.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
%%timeit
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

7.33 ms ± 85.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we're going to leverage Meta's LLaMA 2!

Specifically, we'll be using: `meta-llama/Llama-2-13b-chat-hf`

That's right, a 13B parameter model that we're going to run on *less than* 15GB of GPU RAM.

More information on this model can be found [here](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)

In [20]:
!pip install huggingface-hub -q

In [21]:
from huggingface_hub import notebook_login

notebook_login()

We will be leveraging Tim Dettmer's `bitsandbytes` as well as `accelerate` and `transformers` from Hugging Face to make our model as small as possible. The overall quality of the model is fairly well retained!

In [22]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [23]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [24]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.0,
    max_new_tokens=256
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [25]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [26]:
retriever = vector_store.as_retriever()

In [27]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Now that it's set-up, let's test it out!

In [28]:
qa_with_sources_chain({"query" : "How was Will Ferrell in this movie?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How was Will Ferrell in this movie?',
 'result': ' Based on the reviews, Will Ferrell\'s character was not well received by some of the reviewers. One reviewer described his character as "ruining every scene he was in." Another reviewer mentioned that his board became "superfluous." However, another reviewer found his performance to be enjoyable. Overall, it seems that opinions on Will Ferrell\'s performance in the movie are mixed.',
 'source_documents': [Document(page_content=": 61\nReview_Date: 23 July 2023\nAuthor: agjbull\nRating: 6\nReview_Title: Just a little empty\nReview: I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow. The film was very disjointed. Ryan Gosling as good as he is seemed to old to play Ken and Will Ferrell ruined every scene he was in. I just didn't get it, it seemed hollow artificial and hackneyed. A waste of some great talent. It was predictable without being reassuring and

In [29]:
qa_with_sources_chain({"query" : "Do reviewers consider this movie Kenough?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Do reviewers consider this movie Kenough?',
 'result': ' Based on the reviews provided, it seems that the reviewers have mixed opinions about the movie. Some reviewers, such as eoinageary and hamsterination, have positive reviews and rate the movie 8 and 6 out of 10, respectively. However, other reviewers, such as brianjohnson-20043 and holarsch, have more negative reviews and rate the movie 4 and 6 out of 10, respectively. Therefore, it cannot be determined whether the reviewers consider the movie Kenough or not.',
 'source_documents': [Document(page_content=': 37\nReview_Date: 23 July 2023\nAuthor: eoinageary\nRating: 8\nReview_Title: I am Kenough\nReview: So I went into the movie with little to no expectations and I was pleasantly impressed with the movie overall.\nReview_Url: /review/rw9199947/?ref_=tt_urv', metadata={'source': '/review/rw9199947/?ref_=tt_urv', 'row': 37}),
  Document(page_content=": 9\nReview_Date: 19 July 2023\nAuthor: hamsterination\nRating: 6\nReview

In [30]:
qa_with_sources_chain({"query" : "Did these movies explores themes of existentialism?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Did these movies explores themes of existentialism?',
 'result': ' Based on the reviews, it appears that "Oppenheimer" explores themes of existentialism. The review by sosrivi states that the film is "a reminder to the audience of how obsessed humans are with power and fame," which is a common theme in existentialist literature and film. Additionally, the review by chris_rowe-881-168820 states that the film is "pretty much perfect," which suggests that it effectively explores its themes and ideas.',
 'source_documents': [Document(page_content=": 117\nReview_Date: 20 July 2023\nAuthor: mattlx\nRating: 8\nReview_Title: A decent experience, still not perfect\nReview: The film's abundance of characters, combined with its non-linear narrative approach, might leave you a bit puzzled during your first viewing.\nReview_Url: /review/rw9201962/?ref_=tt_urv", metadata={'source': '/review/rw9201962/?ref_=tt_urv', 'row': 117}),
  Document(page_content=": 135\nReview_Date: 20 July 2023\nA

And with that, we have our Barbie & Oppenheimer Review RAG tool built!

This Notebook is a companion to the event put on by [AIMS](https://www.linkedin.com/company/ai-maker-space/), and [Deci](https://deci.ai/), and is authored by [Chris Alexiuk](https://www.linkedin.com/in/csalexiuk/)